In [4]:
# Full reset of the camera

!echo 'dlinano' | sudo -S systemctl restart nvargus-daemon && printf '\n'

'sudo' 不是內部或外部命令、可執行的程式或批次檔。


In [3]:
# Check device number

!ls -ltrh /dev/video*

In [5]:
# USB Camera (Logitech C270 webcam)

  from jetcam.usb_camera import USBCamera
  camera = USBCamera(width=224, height=224, capture_device=0) # capture_device=0 USB port
  
  camera.running = True
  print("camera created")

IndentationError: unexpected indent (<ipython-input-5-5d7cffda44ff>, line 3)

In [6]:
# Task

 ## Define TASK and CATEGORIES parameters ,in this project it's already been defined,
  ## so just define your project TASK and what CATEGORIES of data you will collect. 
  
  import torchvision.transforms as transforms
  from dataset import ImageClassificationDataset
  
  # set task name 
  TASK = 'thumbs'
  
  # what categories we need
  CATEGORIES = ['thumbs_up', 'thumbs_down']
  
  
  DATASETS = ['A', 'B']
  
  TRANSFORMS = transforms.Compose([    # 用於把一系列變換組合到一起。
  
      # transforms.ColorJitter(brightness=0, contrast=0, saturation=0, hue=0) 修改亮度、對比度和飽和度
      transforms.ColorJitter(0.2, 0.2, 0.2, 0.2), 
      
      # transforms.Resize(size, interpolation=2) 重置影象解析度
      transforms.Resize((224, 224)),
      
      # 轉換為 tensor
      transforms.ToTensor(),
      
      # transforms.Normalize(mean, std) 進行標準化，先減均值，再除以標準差
      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
  ])

  datasets = {}
  
  for name in DATASETS:
      datasets[name] = ImageClassificationDataset(TASK + '_' + name, CATEGORIES, TRANSFORMS)

  print("{} task with {} categories defined".format(TASK, CATEGORIES))
  
  # https://www.itread01.com/content/1546227125.html
  # https://www.itread01.com/content/1546227125.html#10transformsNormalize_120

IndentationError: unexpected indent (<ipython-input-6-1bf4ca1a9851>, line 6)

In [7]:
# Data Collection

## Collect images for your categories with your camera using an iPython widget. 
  import ipywidgets
  import traitlets
  from IPython.display import display
  from jetcam.utils import bgr8_to_jpeg

  # initialize active dataset
  dataset = datasets[DATASETS[0]]

  # unobserve all callbacks from camera in case we are running this cell for second time
  camera.unobserve_all()

  # create image preview
  camera_widget = ipywidgets.Image()
  traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

  # create widgets
  dataset_widget = ipywidgets.Dropdown(options=DATASETS, description='dataset')
  category_widget = ipywidgets.Dropdown(options=dataset.categories, description='category')
  count_widget = ipywidgets.IntText(description='count')
  save_widget = ipywidgets.Button(description='add')

  # manually update counts at initialization
  count_widget.value = dataset.get_count(category_widget.value)

  # sets the active dataset
  def set_dataset(change):
      global dataset
      dataset = datasets[change['new']]
      count_widget.value = dataset.get_count(category_widget.value)
  dataset_widget.observe(set_dataset, names='value')

  # update counts when we select a new category
  def update_counts(change):
      count_widget.value = dataset.get_count(change['new'])
  category_widget.observe(update_counts, names='value')

  # save image for category and update counts
  def save(c):
      dataset.save_entry(camera.value, category_widget.value)
      count_widget.value = dataset.get_count(category_widget.value)
  save_widget.on_click(save)

  data_collection_widget = ipywidgets.VBox([
      ipywidgets.HBox([camera_widget]), dataset_widget, category_widget, count_widget, save_widget
  ])

  # display(data_collection_widget)
  print("data_collection_widget created")
  

IndentationError: unexpected indent (<ipython-input-7-215e672c56da>, line 4)

In [8]:
# Model
import torch
  import torchvision
  
  # Choose GPU device with the statement.
  device = torch.device('cuda')

  # The model is set to the ResNet-18 model.
  # pretrained=True parameter indicates we are loading all the parameter weights for the trained Resnet-18 model.
  model = torchvision.models.resnet18(pretrained=True)
  
  #指维度
  model.fc = torch.nn.Linear(512, len(dataset.categories))

  model = model.to(device)

  model_save_button = ipywidgets.Button(description='save model')
  model_load_button = ipywidgets.Button(description='load model')
  model_path_widget = ipywidgets.Text(description='model path', value='my_model.pth')

  def load_model(c):
      model.load_state_dict(torch.load(model_path_widget.value))
  model_load_button.on_click(load_model)

  def save_model(c):
      torch.save(model.state_dict(), model_path_widget.value)
  model_save_button.on_click(save_model)

  model_widget = ipywidgets.VBox([
      model_path_widget,
      ipywidgets.HBox([model_load_button, model_save_button])
  ])

  # display(model_widget)
  print("model configured and model_widget created")
  

IndentationError: unexpected indent (<ipython-input-8-b6fe310fdf04>, line 3)

In [9]:
# Live Execution

import threading  
  import time
  from utils import preprocess
  import torch.nn.functional as F

  state_widget = ipywidgets.ToggleButtons(options=['stop', 'live'], description='state', value='stop')
  prediction_widget = ipywidgets.Text(description='prediction')
  score_widgets = []
  for category in dataset.categories:
      score_widget = ipywidgets.FloatSlider(min=0.0, max=1.0, description=category, orientation='vertical')
      score_widgets.append(score_widget)

  def live(state_widget, model, camera, prediction_widget, score_widget):
      global dataset
      while state_widget.value == 'live':
          image = camera.value
          preprocessed = preprocess(image)
          output = model(preprocessed)
          output = F.softmax(output, dim=1).detach().cpu().numpy().flatten()
          category_index = output.argmax()
          prediction_widget.value = dataset.categories[category_index]
          for i, score in enumerate(list(output)):
              score_widgets[i].value = score

  def start_live(change):
      if change['new'] == 'live':
          execute_thread = threading.Thread(target=live, args=(state_widget, model, camera, prediction_widget, score_widget))
          execute_thread.start()

  state_widget.observe(start_live, names='value')

  live_execution_widget = ipywidgets.VBox([
      ipywidgets.HBox(score_widgets),
      prediction_widget,
      state_widget
  ])

  # display(live_execution_widget)
  print("live_execution_widget created")

IndentationError: unexpected indent (<ipython-input-9-a778b05f60e5>, line 4)

In [10]:
# Training and Evaluation

BATCH_SIZE = 8

  optimizer = torch.optim.Adam(model.parameters())
  # optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

  epochs_widget = ipywidgets.IntText(description='epochs', value=1)
  eval_button = ipywidgets.Button(description='evaluate')
  train_button = ipywidgets.Button(description='train')
  loss_widget = ipywidgets.FloatText(description='loss')
  accuracy_widget = ipywidgets.FloatText(description='accuracy')
  progress_widget = ipywidgets.FloatProgress(min=0.0, max=1.0, description='progress')

  def train_eval(is_training):
      global BATCH_SIZE, LEARNING_RATE, MOMENTUM, model, dataset, optimizer, eval_button, train_button, accuracy_widget, loss_widget, progress_widget, state_widget

      try:
          train_loader = torch.utils.data.DataLoader(
              dataset,
              batch_size=BATCH_SIZE,
              shuffle=True
          )

          state_widget.value = 'stop'
          train_button.disabled = True
          eval_button.disabled = True
          time.sleep(1)

          if is_training:
              model = model.train()
          else:
              model = model.eval()
          while epochs_widget.value > 0:
              i = 0
              sum_loss = 0.0
              error_count = 0.0
              for images, labels in iter(train_loader):
                  # send data to device
                  images = images.to(device)
                  labels = labels.to(device)

                  if is_training:
                      # zero gradients of parameters
                      optimizer.zero_grad()

                  # execute model to get outputs
                  outputs = model(images)

                  # compute loss
                  loss = F.cross_entropy(outputs, labels)

                  if is_training:
                      # run backpropogation to accumulate gradients
                      loss.backward()

                      # step optimizer to adjust parameters
                      optimizer.step()

                  # increment progress
                  error_count += len(torch.nonzero(outputs.argmax(1) - labels).flatten())
                  count = len(labels.flatten())
                  i += count
                  sum_loss += float(loss)
                  progress_widget.value = i / len(dataset)
                  loss_widget.value = sum_loss / i
                  accuracy_widget.value = 1.0 - error_count / i

              if is_training:
                  epochs_widget.value = epochs_widget.value - 1
              else:
                  break
      except e:
          pass
      model = model.eval()

      train_button.disabled = False
      eval_button.disabled = False
      state_widget.value = 'live'

  train_button.on_click(lambda c: train_eval(is_training=True))
  eval_button.on_click(lambda c: train_eval(is_training=False))

  train_eval_widget = ipywidgets.VBox([
      epochs_widget,
      progress_widget,
      loss_widget,
      accuracy_widget,
      ipywidgets.HBox([train_button, eval_button])
  ])

  # display(train_eval_widget)
  print("trainer configured and train_eval_widget created")

IndentationError: unexpected indent (<ipython-input-10-0cb934326be6>, line 5)

In [ ]:
# Display the Interactive Tool

  # Combine all the widgets into one display
  all_widget = ipywidgets.VBox([
      ipywidgets.HBox([data_collection_widget, live_execution_widget]), 
      train_eval_widget,
      model_widget
  ])

  display(all_widget)
  